In [66]:
from csp import CSP, backtracking_search, AC3, min_conflicts
from utils import argmin_random_tie, count, first
from search import depth_first_graph_search
from itertools import permutations
from itertools import combinations


In [57]:
#class Sumplete(CSP):
#    def __init__(self, variables, domains, neighbors, constraints):
#        super().__init__(variables, domains, neighbors, constraints)
        # You might add additional initialization here if needed.


In [55]:
# TEST CELL

initial_grid = [
    [3, 1, 4, 1],
    [1, 8, 9, 9],
    [5, 3, 2, 8],
    [5, 4, 9, 0]
]

# Extract row and column sums from the initial_grid
row_sums = [initial_grid[i][3] for i in range(3)]  # Target sums for rows
col_sums = [initial_grid[3][i] for i in range(3)]  # Target sums for columns
sums = {'rows': row_sums, 'cols': col_sums}

domains = {}
for r in range(1, 4):
    for c in range(1, 4):
        if initial_grid[r-1][c-1] == 0:  # If the cell is empty
            domains[(r, c)] = list(range(1, 10))  # Can be any digit
        else:
            domains[(r, c)] = [initial_grid[r-1][c-1]]  # The digit is given

# Neighbors: Each cell is a neighbor with the cells in its row and column
neighbors = {v: set() for v in variables}
for r in range(1, 4):
    for c in range(1, 4):
        # Add row neighbors
        neighbors[(r, c)].update([(r, c2) for c2 in range(1, 4) if c2 != c])
        # Add column neighbors
        neighbors[(r, c)].update([(r2, c) for r2 in range(1, 4) if r2 != r])
                                 
# Constraint function
#def sum_constraint(A, a, B, b):
    # Check if A and B are in the same row or column
#    if A[0] == B[0]:  # same row
        # Make sure the row sum does not exceed the target sum
#        row_values = [a if (A[1] == c) else b if (B[1] == c) else initial_grid[A[0]-1][c-1] for c in range(1, 4)]
#        if sum(row_values) != sums[A[0] - 1][3]:
#            return False
#    if A[1] == B[1]:  # same column
        # Make sure the column sum does not exceed the target sum
#        col_values = [a if (A[0] == r) else b if (B[0] == r) else initial_grid[r-1][A[1]-1] for r in range(1, 4)]
#        if sum(col_values) != sums[3][A[1] - 1]:
#            return False

#    return True
# Then, you instantiate the CSP with this constraint function

#def sum_constraint(A, a, B, b):
#    if A[0] == B[0]:  # same row
        # Logic to check row sum...
#        if sum(row_values) != sums['rows'][A[0] - 1]:
#            return False
#    if A[1] == B[1]:  # same column
        # Logic to check column sum...
#        if sum(col_values) != sums['cols'][A[1] - 1]:
#            return False
#    return True
    
def sum_constraint(A, a, B, b):
    if A[0] == B[0]:  # A and B are in the same row
        # Partial sum for the row, considering only A and B
        partial_row_sum = a + b
        if partial_row_sum > sums['rows'][A[0] - 1]:
            return False

    if A[1] == B[1]:  # A and B are in the same column
        # Partial sum for the column, considering only A and B
        partial_col_sum = a + b
        if partial_col_sum > sums['cols'][A[1] - 1]:
            return False

    return True

sumplete_csp = CSP(variables, domains, neighbors, sum_constraint)

# And you run the search algorithm
result = backtracking_search(sumplete_csp)
print(result)

None


In [88]:
def extract_numbers(grid):
    rows = [row[:-1] for row in grid[:-1]]
    columns = [[grid[r][c] for r in range(len(grid)-1)] for c in range(len(grid[0])-1)]
    return rows, columns
    
easy3x3 = [
    [3, 1, 4, 1],
    [1, 8, 9, 9],
    [5, 3, 2, 8],
    [5, 4, 9, 0]
]

rows, columns = extract_numbers(easy3x3)

def valid_combinations(numbers, target_sum):
    all_combs = set()
    for r in range(1, len(numbers) + 1):
        for comb in combinations(numbers, r):
            all_combs.add(comb)
    return [comb for comb in all_combs if sum(comb) == target_sum]


domains = {}
for i in range(len(rows)):  # Assuming a square grid
    domains[f'R{i+1}'] = valid_combinations(rows[i], easy3x3[i][-1])
for i in range(len(columns)):
    domains[f'C{i+1}'] = valid_combinations(columns[i], easy3x3[-1][i])

print(domains)

# For 3x3 puzzle
neighbors = {}
for i in range(1, 4):
    neighbors[f'R{i}'] = [f'C{j}' for j in range(1, 4)]
    neighbors[f'C{i}'] = [f'R{j}' for j in range(1, 4)]

print(neighbors)

def intersection_constraint(var1, value1, var2, value2):
    row_index = int(var1[1]) - 1 if 'R' in var1 else int(var2[1]) - 1
    col_index = int(var2[1]) - 1 if 'C' in var2 else int(var1[1]) - 1

    if row_index < len(value1) and col_index < len(value2):
        print(str(value1[col_index]) + " " + str(value2[row_index]))
        return value1[col_index] == value2[row_index]
    print("Index out of range")
    return True  # Return True if indices are out of range


for var, dom in domains.items():
    print(f"{var}: {dom}")
    
sumplete_csp = CSP(variables=list(domains.keys()), domains=domains, neighbors=neighbors, constraints=intersection_constraint)
solution = backtracking_search(sumplete_csp)
print("Solution:\n", solution)


{'R1': [(1,)], 'R2': [(1, 8), (9,)], 'R3': [(5, 3)], 'C1': [(5,)], 'C2': [(1, 3)], 'C3': [(9,)]}
{'R1': ['C1', 'C2', 'C3'], 'C1': ['R1', 'R2', 'R3'], 'R2': ['C1', 'C2', 'C3'], 'C2': ['R1', 'R2', 'R3'], 'R3': ['C1', 'C2', 'C3'], 'C3': ['R1', 'R2', 'R3']}
R1: [(1,)]
R2: [(1, 8), (9,)]
R3: [(5, 3)]
C1: [(5,)]
C2: [(1, 3)]
C3: [(9,)]
Comparing 5 (Row) and 1 (Column)
Comparing 5 (Row) and 8 (Column)
Index out of range - Returning True
Comparing 5 (Row) and 1 (Column)
Index out of range - Returning True
Index out of range - Returning True
Solution:
 None
